# Исследование надежности заемщиков

Заказчик — кредитный отдел банка. 

**Цель исследования:** Определить, какие социально-демографические факторы влияют на факт погашения кредита в срок. 
К таким факторам относят: количество детей у заёмщика, его семейное положение, уровень доходов и цели кредитования.

**Ход исследования:**
1. Обзор входных данных от банка (статистика о платёжеспособности клиентов.) 
2. Предобработка данных. 
3. Проверка гипотез о влиянии социально-демографических факторов на своевременность погашения кредита в срок.

**Использованные данные**:

- количество детей в семье
- общий трудовой стаж в днях
- возраст клиента в годах
- уровень образования клиента
- идентификатор уровня образования
- семейное положение
- идентификатор семейного положения
- пол клиента
- тип занятости
- имел ли задолженность по возврату кредитов
- ежемесячный доход
- цель получения кредита

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Ход исследования

### Шаг 1. Обзор данных

In [81]:
import pandas as pd

In [82]:
try:
    df = pd.read_csv('/datasets/data.csv')
except:
    df = pd.read_csv('data.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.1 Заполнение пропусков

In [83]:
print('Пропущенных значений в столбце "Общий трудовой стаж в днях":', len(df[df['days_employed'].isna()]))
print('Пропущенных значений в столбце "Ежемесячный доход":', len(df[df['total_income'].isna()]))

print('Доля пропущенных значений в столбце "Общий трудовой стаж в днях":', len(df[df['days_employed'].isna()]) / len(df['days_employed']))
print('Пропущенных значений в столбце "Ежемесячный доход":', len(df[df['total_income'].isna()]) / len(df['total_income']))

Пропущенных значений в столбце "Общий трудовой стаж в днях": 2174
Пропущенных значений в столбце "Ежемесячный доход": 2174
Доля пропущенных значений в столбце "Общий трудовой стаж в днях": 0.10099883855981417
Пропущенных значений в столбце "Ежемесячный доход": 0.10099883855981417


Обнаружены пропущенные значения в двух столбцах: days_employed и total_income . Количество таких значений в каждом столбце - 2174. Доля пропущенных значений в обоих случаях - 0,1 (ок. 10 %) Возможная причина отсутствия данных - техническая, например, сбой при переносе данных из одной базы в другую.

Возможное решение замены отсутствующих данных - заполнение медианным значением, среднее в данном случае не подойдет, т.к. при больших крайних значениях стажа или дохода будет влияние на среднее, а медиана позволит это влияние устранить.

Посмотрим на данные подробнее.

### Шаг 2.2 Проверка данных на аномалии и исправления. Изменение типов данных.

In [84]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


В столбце с трудовым стажем обнаружены отрицательные значения. Скорее всего это ошибка ввода (когда данные в соседних ячейках перепутаны местами, а при вычитании в финале получили отрицательные значения.) Исправим эти данные методом abs().

In [85]:
df['days_employed'] = df['days_employed'].abs()
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Также в столбце с трудовым стажем мы видим аномальные значения значения, а именно в строке с индексом 4 - более 340 т. дней. Понятно, что это не выброс, а именно ошибка. Для начала надо понять, какое количество строк у нас с похожими значениями, а потом примем решение, как их исправлять.

In [86]:
print('Максимальный возраст клиента банка:', df['dob_years'].max())
days_employed_treshold = (75 - 14) * 247
print('Максимально возможный стаж:', days_employed_treshold, 'дней')

days_employed_errors = df.loc[df['days_employed'] > days_employed_treshold]
print('Количество возможных ошибок в столбце со стажем:', days_employed_errors['days_employed'].count())

days_employed_errors_sorted = days_employed_errors.sort_values(by='days_employed', ascending=False)
days_employed_errors_sorted.tail(5)

Максимальный возраст клиента банка: 75
Максимально возможный стаж: 15067 дней
Количество возможных ошибок в столбце со стажем: 3458


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
7731,0,15618.063786,64,среднее,1,женат / замужем,0,F,компаньон,0,296525.358574,высшее образование
15675,0,15410.040779,65,высшее,0,женат / замужем,0,F,сотрудник,0,188800.068859,покупка жилой недвижимости
19788,0,15267.541183,56,среднее,1,женат / замужем,0,M,сотрудник,0,122976.371341,приобретение автомобиля
16868,0,15193.032201,57,среднее,1,женат / замужем,0,F,госслужащий,0,194993.882911,строительство недвижимости
5581,0,15079.216069,55,среднее,1,женат / замужем,0,F,госслужащий,0,178761.373413,операции с жильем


Попробуем найти границу и дапазон аномальных значений. Максимальный возраст заемщиков, судя по данным - 75 лет. Для этого возраста максимально возможный стаж - 15067 дней (трудовой возраст в России - с 14 лет, а количество рабочих дней в году - в среднем 247 дней.)

Посмотрим, что происходит в таблице после порога в 15067 дней стажа. Всего таких строк 3458, при этом мы получили небольшое количество строк в диапазоне от 15079 до 18388 дней, а потом резкий скачок к значению в 328728 дней.

Если установить уже эту границу (328728), то у нас уже будет 3445 строк. Диапазон аномальных значений - от 328728 до 401755.

In [87]:
print('Количество аномалий в столбце со стажем:', df['days_employed'].loc[df['days_employed'] >= 328728].count())

Количество аномалий в столбце со стажем: 3445


Что делать c такими аномальными значениями? Посмотрим, есть ли что-то общее для таких строк? 
Например, какая занятость у таких заемщиков?

In [88]:
occupation_anomal_days = df.loc[df['days_employed'] >= 328728]
print(occupation_anomal_days['income_type'].value_counts())

пенсионер      3443
безработный       2
Name: income_type, dtype: int64


Как видим, подавляющее число заемщиков с аномальными значениями стажа - пенсионеры. Это значит, что при исправлении значений их стажа, мы должны получить данные, которые будут расположены между средним и максимальными значениями стажа в таблице без учета аномальных данных (в силу возраста этих заемщиков.) Поэтому мы видим **2 сценария**: либо предположить, что ошибка в данных произошла из-за сдвига плавающей запятой (т.е. стаж по факту меньше в 100 раз, нежели указанный.) Либо в датасете указан другой тип данных - не дни, а, например, часы. Тогда аномальный стаж надо разделить на количество часов, которые входят в "трудовой день".

In [89]:
print('Медиана стажа без учета аномалий:')
print(df.loc[df['days_employed'] < 328728, 'days_employed'].median())
print('Среднее стажа без учета аномалий:')
print(df.loc[df['days_employed'] < 328728, 'days_employed'].mean())

Медиана стажа без учета аномалий:
1630.0193809778218
Среднее стажа без учета аномалий:
2353.015931998879


- В **первом сценарии** у нас получатся значения в примерном диапазоне от 3000 до 4000 дней стажа. Как мы видим, это больше и среднего и медианы выборки без аномалий, но в несколько раз меньше максимальных значений (15-18 т. рабочих дней). В принципе выглядит реалистично. 
- Во **втором** не очень понятно, как действовать: делить аномальные данные на 8 (длительность рабочего дня при графике 5/2), на 12 (график 2/2) или вообще на 24 (график "сутки через трое".) Если брать первые 2 действия, то количество дней опять станет малореалистичным. Если идти по пути "суток", мы получаем значения, близкие к макисимальным в выборке без аномалий (от 14 т. дней до 17 т.), НО представить себе человека, который смог отбработать такое количество дней в графике "сутки через трое" мы не сможем, это фантастика. Итого, придерживаемся сценария, при котором ошибка возникла из-за "плавающей" точки, которая должна быть в другом месте у аномальных данных. 
- **Исправляем** аномальные данные через деление на 100.

In [90]:
df.loc[df['days_employed'] >= 328728, 'days_employed'] = df[df['days_employed'] >= 328728]['days_employed']/100
df.sort_values(by='days_employed', ascending=False).head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью
4299,0,17615.563266,61,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья
7329,0,16593.472817,60,высшее,0,женат / замужем,0,F,сотрудник,0,124697.846781,заняться высшим образованием
17838,0,16264.699501,59,среднее,1,женат / замужем,0,F,сотрудник,0,51238.967133,на покупку автомобиля
16825,0,16119.687737,64,среднее,1,женат / замужем,0,F,сотрудник,0,91527.685995,покупка жилой недвижимости


Аномалии (стаж длительностью более 328728 дней) заменены. Остались пропуски. Устраним им их, вставив медианные значения стажа. Также переведем данные стажа в целочисленный тип.

In [91]:
days_employed_median = df['days_employed'].median()
print('Медиана трудового стажа:', days_employed_median)

df['days_employed'] = df['days_employed'].fillna(days_employed_median)
df['days_employed'] = df['days_employed'].astype(int)

df.info()

Медиана трудового стажа: 2194.220566878695
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int32  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 1.9+ MB


Далее поработаем с пропусками в столбце с ежемесячным доходом. Посмотрим характеристики данных этого столбца.

In [92]:
df['total_income'].describe().astype(int)

count      19351
mean      167422
std       102971
min        20667
25%       103053
50%       145017
75%       203435
max      2265604
Name: total_income, dtype: int32

Среднее от медианы отличается не очень сильно. При этом довольно большая разница между минимальным и максимальным значениями и большое стандартное отклонение. Может, строки с отсутствующими значениями дохода можно сгруппировать по типу занятости? Тогда можно будет взять не общую медиану, а использовать медиану какой-нибуть одной группы занятых, если эта группа будет доминировать в пустых строках. 

Посмотрим соотношение типа занятости в заполненных и пустых строках дохода.

In [93]:
total_income_empty = df.loc[df['total_income'].isnull()]
print('Типы занятых в пустых строках дохода:')
print(total_income_empty['income_type'].value_counts())

print('Типы занятых в заполненых строках дохода:')
print(df['income_type'].value_counts())

Типы занятых в пустых строках дохода:
сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64
Типы занятых в заполненых строках дохода:
сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64


Видно, что в обоих случаях группы занятых представлены очень похожим образом - одинаковый порядок (по убыванию) и схожее соотношение групп занятых. Так что мы принимаем решение заполнять пустые строки дохода общей медианой. Также переведем данные дохода в целочисленный тип.

In [94]:
total_income_median = df['total_income'].median()
df['total_income'] = df['total_income'].fillna(total_income_median)
df['total_income'] = df['total_income'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


Все пропуски заполнены. Посмотрим, есть ли аномалии в столбце с возрастом.

In [95]:
print(df['dob_years'].max())
print(df['dob_years'].min())

75
0


Максимальное значение вопросов не вызывает, но в столбце с возрастом есть значения "0". Посмотрим, много ли значений возраста менее 14 лет (14 лет - трудовой возраст по законам России.)

In [96]:
df_14 = df.loc[df['dob_years'] < 14]
print('Количество несовершеннолетних (возраст и количество):', df_14['dob_years'].value_counts())

Количество несовершеннолетних (возраст и количество): 0    101
Name: dob_years, dtype: int64


Примерно у 0,5% заемщиков не указан возраст (точнее указано значение "0"). Судя по задачам исследования эти данные не так важны для проверки гипотез, поэтому оставим эти ячейки без изменений. Далее посмотрим данные в столбце с количеством детей.

In [97]:
print(df['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Переведем отрицательные значения к абсолютной величине. Также обратим внимание на 76 случаев, когда у заемщика указано 20 детей. Это может быть ошибкой (например, верное количество - 2.0), а может быть и верным значением: такие случаи и вправду редки, к тому же речь может идти о приемных детях. Плюс такие заемщики с бОльшей вероятностью могут быть заинтересованы в услугах банка, поэтому их доля среди всех заемщиков намного больше, чем доля многодетных в статистике российских семей.

In [98]:
df['children'] = df['children'].abs()
df['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

Приведем написание типа образования к нижнему регистру

In [99]:
df['education'] = df['education'].str.lower()
print(df['education'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


Также мы проверили все столбцы с текстовыми данными (при помощи метода unique()). В столбце "education" обнаружена разница в регистре написания типа образования. Эта разница устранена, и все данные написаны в строчном режиме. Это позволило избежать неявных дублей. Причина возникновения неявных дублей - скорее всего ошибка ввода. Еще одна возможная причина - сведение данных из разных источников, где типы образования отличались друг от друга в плане использования строчных и прописных букв.

### Шаг 2.3. Удаление дубликатов.

Далее мы посчитаем количество строк-дубликатов, и удалим их из таблицы. Возможные причины пояления дубликатов - сброс датасета в несколько этапов, в этом случае одинаковые строки могут не заменяться, а копироваться. Способы поиска и удаления дубликатов - стандартые методы библиотеки pandas.

In [100]:
print('Количество строк-дубликатов:', df.duplicated().sum())

df = df.drop_duplicates().reset_index(drop=True)

print('Количество строк-дубликатов после удаления:', df.duplicated().sum())

Количество строк-дубликатов: 71
Количество строк-дубликатов после удаления: 0


### Шаг 2.4. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим новые дейтафреймы для данных по образованию и семейному статусу, они послужат "словарями", к которым можно будет обращаться по идентификатору. Далее удаляем столбцы 'education' и 'family_status' из исходного дейтафрейма.

In [101]:
education_dict = df[['education_id', 'education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)

family_status_dict = df[['family_status_id', 'family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)


display(education_dict.head())
display(family_status_dict.head())

del df['education']
del df['family_status']

df.columns

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Index(['children', 'days_employed', 'dob_years', 'education_id',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose'],
      dtype='object')

### Шаг 2.5. Категоризация дохода.

В основной таблице создадим столбец 'total_income_category', в котором клиентов банка распределили по категориям в зависимости от уровня дохода.

In [102]:
def total_income_category(income):
    if income <= 30000:
        return 'E'
    if income <= 50000:
        return 'D'
    if income <= 200000:
        return 'C'
    if income <= 1000000:
        return 'B'
    return 'A'

df['total_income_category'] = df['total_income'].apply(total_income_category)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,3402,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.6. Категоризация целей кредита.

Посмотрим, какие цели преследуют заемщики банка.

In [103]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

В основной таблице создадим столбец 'purpose_category', в котором клиентов банка распределили по категориям в зависимости от целей кредитования.

In [104]:
def purpose_category(purpose):
    if 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'недв' in purpose:
        return 'операции с недвижимостью'
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'образ' in purpose:
        return 'получение образования'
    return 'проведение свадьбы'

df['purpose_category'] = df['purpose'].apply(purpose_category)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,3402,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


### Ответы на вопросы.

Посмотрим, каков процент возврата кредита среди 4-х групп категорий заемщиков: 
- категории с детьми/без детей
- в зависимости от семейного положения
- дохода
- целей кредита

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [105]:
children_pivot = df.pivot_table(index='children', columns='debt', values='days_employed', aggfunc='count')
children_pivot['debt_percentage'] = (children_pivot[1]/(children_pivot[1] + children_pivot[0]))*100
children_pivot

debt,0,1,debt_percentage
children,,,
0,13028.0,1063.0,7.543822
1,4410.0,445.0,9.165808
2,1858.0,194.0,9.454191
3,303.0,27.0,8.181818
4,37.0,4.0,9.756098
5,9.0,NaN,NaN
20,68.0,8.0,10.526316


##### Вывод 1:

Судя по распределению процентов возврата кредита, **небольшая закономерность** есть: **бездетные** возвращают кредит в срок чаще всего (и при этом берут кредит намного **чаще** остальных, ок 66% кредиторов - бездетные.) 

Если же говорить о заемщиках с разным количеством детей, то **прямой зависимости не видно**. Можно уверенно сказать, что чаще проблемы с возвратом у многодетных заемщиков (20 детей), но таких примеров в принципе немного (и то, если предположить, что это не ошибочные данные.) Очень хорошо возвращают заемщики с 5ю детьми, у них 100% возврата вовремя, но таких в абсолютных цифрах еще меньше, чем тех, у кого 20 детей. При этом **небольшие значения** просрочки у заемщиков с 3-мя детьми.

##### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [106]:
family_status_pivot = df.pivot_table(index='family_status_id', columns='debt', values='days_employed', aggfunc='count')
family_status_pivot['debt_percentage'] = (family_status_pivot[1]/(family_status_pivot[1] + family_status_pivot[0]))*100
family_status_pivot = family_status_pivot.merge(family_status_dict, on='family_status_id', how='left')
family_status_pivot

,family_status_id,0,1,debt_percentage,family_status
0,0,11408,931,7.545182,женат / замужем
1,1,3763,388,9.347145,гражданский брак
2,2,896,63,6.569343,вдовец / вдова
3,3,1110,85,7.112971,в разводе
4,4,2536,274,9.750890,Не женат / не замужем


##### Вывод 2:

**Меньше** всего задерживают выплату кредита **вдовцы/вдовы** и **разведенные** заемщики (видимо, они более тщательно следят за финансами.) Не замужние/ не женатые чаще других не соблюдают сроки выплат. 

Так что зависимость все-таки **наблюдается**.

##### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [107]:
income_pivot = df.pivot_table(index='total_income_category', columns='debt', values='days_employed', aggfunc='count')
income_pivot['debt_percentage'] = (income_pivot[1]/(income_pivot[1] + income_pivot[0]))*100
income_pivot['total_income_description'] = ['Доход от 1 миллиона', 'От 200к до 1М', 'От 50к до 200к', 'От 30к до 50к', 'Менее 30к']
income_pivot

debt,0,1,debt_percentage,total_income_description
total_income_category,,,,
A,23,2,8.000000,Доход от 1 миллиона
B,4685,356,7.062091,От 200к до 1М
C,14656,1360,8.491508,От 50к до 200к
D,329,21,6.000000,От 30к до 50к
E,20,2,9.090909,Менее 30к


##### Вывод 3:

В этой категории самый **большой разброс** процентов невозврата. Чаще всего вовремя расплачиваются заемщики с **доходом 30-50 т.р.** А вот те, кто мало зарабатывают (менее 30 т.р.) чаще задерживают, но таких заемщиков мало, возможно банк неохотно дает кредиты этой группе. Довольно часто **задерживает** возврат самая многочисленная группа заемщиков с доходом в 50-200 т.р. (те, кого в России считают средним классом.) 

Можем утверждать, что **уровень дохода влияет на вероятность возврата кредита в срок**.

##### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

In [108]:
purpose_pivot = df.pivot_table(index='purpose_category', columns='debt', values='days_employed', aggfunc='count')
purpose_pivot['debt_percentage'] = (purpose_pivot[1]/(purpose_pivot[1] + purpose_pivot[0]))*100
purpose_pivot

debt,0,1,debt_percentage
purpose_category,,,
операции с автомобилем,3903,403,9.359034
операции с недвижимостью,10029,782,7.233373
получение образования,3643,370,9.220035
проведение свадьбы,2138,186,8.003442


##### Вывод 4:

В случае **операций с недвижимостью** заемщики чаще всего возвращают кредит в срок (хотя сам по себе процент невозврата не самый маленький из всех категорий заемщиков.) А вот те, кто взяли кредиты на покупку автомобиля и получение образования примерно в 10% случаев **задерживали** выплаты. Зависимость, хоть не **небольшая**, но видима.

## Общий вывод:

Мы изучили доли возврата/задержки возврата кредитов для разных категорий заёмщиков и обнаружили, что социально-демографические факторы хоть и в разной степени, но влияют на ответственное поведение клиентов банка. 

Если рискнуть обобщить эти данные, то можно составить портреты рискованного и ответственного заёмщиков. 
- **задержит выплату**: многодетный (или наоборот с 1-2 детьми) заемщик в гражданском браке с доходом от 50 до 200 т.р., решивший сменить или купить автомобиль. 
- **вернет вовремя**: бездетный вдовец с доходом 30-50 т.р., решивший обменять или купить недвижимость.